# Computational Theory Assessment

## Problem 1

In [3]:
import numpy as np

def parity(x, y, z):
    """
    Parity function: x ⊕ y ⊕ z
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of XOR operation
    """
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    return np.uint32(x ^ y ^ z)

In [ ]:
def ch(x, y, z):
    """
    Choice function: (x ∧ y) ⊕ (¬x ∧ z)
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of choice operation
    """
    # Convert inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    
    # For each bit: if x bit is 1, choose y bit; if x bit is 0, choose z bit
    return np.uint32((x & y) ^ (~x & z))

### Choice Function Explanation

The **Choice (Ch)** function acts as a bitwise multiplexer. It examines each bit position in `x`:
- If the bit in `x` is 1, it selects the corresponding bit from `y`
- If the bit in `x` is 0, it selects the corresponding bit from `z`

This function is crucial in SHA-256's compression function, providing conditional bit selection based on the first input.

**Mathematical representation:** `Ch(x,y,z) = (x ∧ y) ⊕ (¬x ∧ z)`

In [8]:
def maj(x, y, z):
    """
    Majority function: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    For each bit position, returns the majority bit (the bit that appears most frequently).
    
    Args:
        x, y, z: 32-bit integers
        
    Returns:
        32-bit integer result of majority operation
    """

    # Convert inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Return the majority bit for each position
    # If at least 2 of 3 bits are 1, result bit is 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

### Majority Function Explanation

The **Majority (Maj)** function implements democratic bit selection. For each bit position, it returns 1 if at least two of the three input bits are 1, otherwise it returns 0.

This function provides resistance against certain cryptographic attacks by ensuring that no single input can completely control the output.

**Mathematical representation:** `Maj(x,y,z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)`

In [10]:
def rotr(value, positions):
    """
    Right rotate operation for 32-bit integers
    
    Args:
        value: 32-bit integer to rotate
        positions: number of positions to rotate right
        
    Returns:
        32-bit integer result of right rotation
    """
    # Ensure 32-bit unsigned integer
    value = np.uint32(value)
    positions = positions % 32  # Handle positions > 32
    
    # Right rotation: move bits right, wrap around to left
    return np.uint32((value >> positions) | (value << (32 - positions)))

In [11]:
def shr(value, positions):
    """
    Right shift operation for 32-bit integers
    
    Args:
        value: 32-bit integer to shift
        positions: number of positions to shift right
        
    Returns:
        32-bit integer result of right shift
    """
    # Ensure 32-bit unsigned integer and perform logical right shift
    value = np.uint32(value)
    return np.uint32(value >> positions)

### Helper Functions Explanation: Rotation and Shifting

These helper functions implement fundamental bit manipulation operations:

- **ROTR (Right Rotate):** Moves bits to the right, with bits that fall off the right end wrapping around to the left end
- **SHR (Right Shift):** Moves bits to the right, filling the left end with zeros

These operations are building blocks for the sigma functions and provide the bit diffusion necessary for cryptographic security.

In [12]:
def sigma0(x):
    """
    σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ SHR¹⁰(x)
    
    Lowercase sigma function used in SHA-256 message schedule.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of σ₀ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations and shift as specified in SHA-256 standard
    rotr_7 = rotr(x, 7)    # Right rotate by 7 positions
    rotr_18 = rotr(x, 18)  # Right rotate by 18 positions
    shr_3 = shr(x, 3)      # Right shift by 3 positions
    
    # XOR all three results together
    return np.uint32(rotr_7 ^ rotr_18 ^ shr_3)

In [13]:
def sigma1(x):
    """
    σ₁²⁵⁶(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    Lowercase sigma function used in SHA-256 message schedule.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of σ₁ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations and shift as specified in SHA-256 standard
    rotr_17 = rotr(x, 17)  # Right rotate by 17 positions
    rotr_19 = rotr(x, 19)  # Right rotate by 19 positions
    shr_10 = shr(x, 10)    # Right shift by 10 positions
    
    # XOR all three results together
    return np.uint32(rotr_17 ^ rotr_19 ^ shr_10)

### Message Schedule Functions (σ₀ and σ₁)

The lowercase sigma functions are used in SHA-256's **message schedule** to expand the 16-word message block into 64 words:

- **σ₀(x):** Combines right rotations by 7 and 18 positions with a right shift by 3 positions
- **σ₁(x):** Combines right rotations by 17 and 19 positions with a right shift by 10 positions

These functions ensure that each bit of the input influences multiple bits of the output, providing excellent diffusion properties essential for cryptographic security.

In [14]:
def Sigma0(x):
    """
    Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Uppercase Sigma function used in SHA-256 compression function.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of Σ₀ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations as specified in SHA-256 standard
    rotr_2 = rotr(x, 2)    # Right rotate by 2 positions
    rotr_13 = rotr(x, 13)  # Right rotate by 13 positions
    rotr_22 = rotr(x, 22)  # Right rotate by 22 positions
    
    # XOR all three results together
    return np.uint32(rotr_2 ^ rotr_13 ^ rotr_22)


In [15]:
def Sigma1(x):
    """
    Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Uppercase Sigma function used in SHA-256 compression function.
    
    Args:
        x: 32-bit integer
        
    Returns:
        32-bit integer result of Σ₁ operation
    """
    # Convert to 32-bit unsigned integer
    x = np.uint32(x)
    
    # Apply rotations as specified in SHA-256 standard
    rotr_6 = rotr(x, 6)    # Right rotate by 6 positions
    rotr_11 = rotr(x, 11)  # Right rotate by 11 positions
    rotr_25 = rotr(x, 25)  # Right rotate by 25 positions
    
    # XOR all three results together
    return np.uint32(rotr_6 ^ rotr_11 ^ rotr_25)


### Compression Functions (Σ₀ and Σ₁)

The uppercase Sigma functions are used in SHA-256's **compression function** during the main hashing rounds:

- **Σ₀(x):** Combines right rotations by 2, 13, and 22 positions
- **Σ₁(x):** Combines right rotations by 6, 11, and 25 positions

Unlike the message schedule functions, these use only rotations (no shifts), ensuring that no information is lost. They provide non-linear transformations that make the hash function resistant to various cryptographic attacks.

In [16]:
# Test all implemented functions with appropriate examples
print("SHA-256 Function Implementation Tests")
print("=" * 40)

# Define test values
test_x = 0x12345678
test_y = 0x9ABCDEF0
test_z = 0x87654321

print(f"Test inputs:")
print(f"  x = 0x{test_x:08X}")
print(f"  y = 0x{test_y:08X}")
print(f"  z = 0x{test_z:08X}")
print()

# Test three-input functions
print("Three-input functions:")
print(f"  Parity(x,y,z) = 0x{parity(test_x, test_y, test_z):08X}")
print(f"  Ch(x,y,z)     = 0x{ch(test_x, test_y, test_z):08X}")
print(f"  Maj(x,y,z)    = 0x{maj(test_x, test_y, test_z):08X}")
print()

# Test single-input sigma functions
print("Message schedule functions:")
print(f"  σ₀(x) = 0x{sigma0(test_x):08X}")
print(f"  σ₁(x) = 0x{sigma1(test_x):08X}")
print()

# Test single-input Sigma functions
print("Compression functions:")
print(f"  Σ₀(x) = 0x{Sigma0(test_x):08X}")
print(f"  Σ₁(x) = 0x{Sigma1(test_x):08X}")

SHA-256 Function Implementation Tests
Test inputs:
  x = 0x12345678
  y = 0x9ABCDEF0
  z = 0x87654321

Three-input functions:
  Parity(x,y,z) = 0x0FEDCBA9
  Ch(x,y,z)     = 0x97755771
  Maj(x,y,z)    = 0x92345670

Message schedule functions:
  σ₀(x) = 0xE7FCE6EE
  σ₁(x) = 0xA1F78649

Compression functions:
  Σ₀(x) = 0x66146474
  Σ₁(x) = 0x3561ABDA


## Problem 2

In [1]:
import numpy as np
import math

def primes(n):
    """
    Generate the first n prime numbers using the Sieve of Eratosthenes algorithm.
    
    Args:
        n: Number of prime numbers to generate
        
    Returns:
        List of the first n prime numbers
    """
    if n <= 0:
        return []
    
    # Estimate upper bound for the nth prime using prime number theorem
    # For n >= 6, the nth prime is less than n * ln(n * ln(n))
    if n < 6:
        upper_bound = 15  # Safe bound for first few primes
    else:
        upper_bound = int(n * math.log(n * math.log(n))) + 100
    
    # Initialize sieve array - True means potentially prime
    sieve = [True] * (upper_bound + 1)
    sieve[0] = sieve[1] = False  # 0 and 1 are not prime
    
    # Apply Sieve of Eratosthenes
    for i in range(2, int(math.sqrt(upper_bound)) + 1):
        if sieve[i]:  # i is prime
            # Mark all multiples of i as composite
            for j in range(i * i, upper_bound + 1, i):
                sieve[j] = False
    
    # Collect first n primes
    prime_list = []
    for i in range(2, upper_bound + 1):
        if sieve[i]:
            prime_list.append(i)
            if len(prime_list) == n:
                break
    
    return prime_list

## Problem 3

## Problem 4

## Problem 5